In [2]:
import torch
import torch.nn.functional as F

%matplotlib inline

In [4]:
words = open("names.txt", "r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

In [6]:
chars = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
# build the dataset

block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size

    for ch in w + ".":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print("".join(itos[i] for i in context), "--->", itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [8]:
C = torch.randn((27, 2))  # embed 27 characters in a 2-dim vector space
# essentially, if there is a 2-dim space, then each one of the 27 chars can be represented as a point in that space.

tensor([[ 0.0814, -0.6140],
        [-0.0959, -0.5584],
        [-1.1340,  0.2147],
        [ 0.3147,  1.0011],
        [ 0.5820,  0.0830],
        [ 0.1351, -0.4158],
        [-0.2471, -1.4509],
        [ 0.0592,  0.9711],
        [-1.0929,  0.2024],
        [-1.1358, -1.1530],
        [ 2.0353,  0.9457],
        [ 1.4621,  0.3522],
        [ 1.9702,  0.1762],
        [-0.4011,  1.2130],
        [-1.5304,  0.2306],
        [-0.3068,  1.1294],
        [-1.8209, -1.2288],
        [ 0.4419, -0.9674],
        [ 0.2075, -0.3031],
        [ 1.4885, -0.4311],
        [ 0.5762,  0.0565],
        [-1.5107, -0.2885],
        [-0.4380, -0.0329],
        [-2.0340, -1.1478],
        [-0.4024, -1.6616],
        [ 0.1336, -1.1604],
        [ 1.3336, -1.7852]])

In [10]:
emb = C[X]  # embeddings of individual starting data points

In [11]:
# construct hidden layer.
# 6 - inputs cuz 3 * 2 -> 3 of 2-dim embeddings
W1 = torch.randn((6, 100))  # 100 neurons for the layer
b1 = torch.randn(100)

In [17]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [19]:
h.shape

torch.Size([32, 100])

In [21]:
# output layer

W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [22]:
logits = h @ W2 + b2

In [23]:
logits

tensor([[ 3.7385e+00,  2.0122e+00,  3.5305e+00, -5.0133e-01, -2.1512e+00,
          1.0750e+01,  3.1232e+00, -7.7684e+00, -3.0627e+00,  1.1895e+01,
         -4.9876e+00,  9.8331e-01, -1.7642e+00, -4.2013e-01,  1.4515e+00,
         -6.9971e+00, -2.4225e+00,  8.9052e+00, -8.5158e+00, -4.1267e-01,
          3.8589e+00, -5.1449e+00,  5.1209e+00, -6.8862e+00, -6.0200e+00,
          2.4966e+00, -4.8985e+00],
        [ 2.5061e+00,  2.7043e+00,  3.5535e+00, -1.4629e+00, -8.9170e-01,
          9.1412e+00,  1.7817e+00, -7.9417e+00, -3.5714e+00,  1.2567e+01,
         -6.9508e+00,  2.0888e-01, -1.4832e+00, -7.4333e-02,  6.2971e-01,
         -8.1726e+00, -2.1720e+00,  7.9287e+00, -7.5698e+00, -1.1164e-01,
          6.0629e+00, -5.7024e+00,  3.1497e+00, -6.7898e+00, -5.8303e+00,
          1.4900e+00, -3.1915e+00],
        [ 1.4843e-01, -1.4037e+01,  1.0286e+01, -1.8062e+01,  1.3029e+01,
         -1.0030e+01, -7.0786e+00, -7.9898e+00, -1.8429e+00, -4.7375e+00,
         -8.4178e+00, -4.8414e+00,  7.58

In [24]:
counts = logits.exp()

In [25]:
prob = counts / counts.sum(1, keepdims=True)

In [26]:
prob.shape

torch.Size([32, 27])

In [29]:
loss = -prob[torch.arange(32), Y].log().mean()

In [30]:
loss

tensor(14.1654)

In [31]:
# rearrange

In [32]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)  # embed 27 characters in a 2-dim vector space
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [33]:
sum(p.nelement() for p in parameters)

3481

In [36]:
for p in parameters:
    p.requires_grad = True

In [39]:
# forward pass

for _ in range(1000):
    emb = C[X]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2

    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = prob[torch.arange(32, Y)]

    loss = F.cross_entropy(logits, Y)
    # print(loss.item())

    # backward pass

    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item())

0.2556043565273285
